In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import time
import warnings
import logging
import sqlalchemy
from sqlalchemy import create_engine, MetaData
warnings.filterwarnings('ignore')
from pprint import pprint
from config import *

logger = logging.Logger('catch_all')

# Stock data url connection requirements
url = "https://www.alphavantage.co/query?"
function = "function=TIME_SERIES_DAILY"
output="&outputsize=full"
key = f"&apikey={stock_api_key}"

# PostgreSQL connection and creating session
connect_str = 'postgresql://postgres:'+db_pass+'@'+db_host+':'+db_port+'/'+db_name
engine = create_engine(connect_str)
connection = engine.connect()

In [ ]:
surl = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [ ]:
tables = pd.read_html(surl)
# tables

In [ ]:
df = tables[0]
df.columns= ['symbol', 'security', 'SEC_filings', 'gics_sector', 'gics_sub_industry','headquarters_location', 'date_first_added', 'cik', 'founded']
del df['SEC_filings']
df.head()

In [ ]:
# df.count()

In [ ]:
data_type = {"symbol": sqlalchemy.types.VARCHAR(length=10), 
             "security": sqlalchemy.types.VARCHAR(),
             "gics_sector": sqlalchemy.types.VARCHAR(),
             "gics_sub_industry": sqlalchemy.types.VARCHAR(),
             "headquarters_location	": sqlalchemy.types.VARCHAR(),
             # "date_first_added": sqlalchemy.types.DateTime(),
             "date_first_added": sqlalchemy.types.VARCHAR(length=50),
             "cik": sqlalchemy.types.Integer(),
             "founded": sqlalchemy.types.VARCHAR(length=50)
            }

In [ ]:
df.to_sql(name="sp500", con=engine, if_exists='replace', index=False, dtype=data_type)

In [ ]:
import psycopg2
conn = psycopg2.connect(database = db_name, user = "postgres", password=db_pass, host=db_host, port=db_port)
command = (
"""
create table if not exists stock_daily (
    stock_date  timestamp,
    open double precision,
    high double precision,
    low double precision,
    close double precision,
    volume bigint,
    stock_symbol varchar(10)
)
"""
)
cur = conn.cursor()
cur.execute(command)
conn.commit()
# cur.close()
# conn.close()


In [3]:
pending_symbol = pd.read_sql("select distinct symbol from sp500 where not exists (select 1 from stock_daily where stock_symbol = symbol)", connection)
pending_symbol

,symbol
0,BKB


In [4]:
# up to 5 API requests per minute and 500 requests per day
err_cnt = 0
rec_cnt = 0
start_time = time.time()
etime = time.time() + 60
for index, row in pending_symbol.iterrows():
    rec_cnt += 1;
    print(index, row['symbol'], rec_cnt)
    try:
        print(f"Retrieving Stock data from Alphavantage website for the symbol {row['symbol']}")
        symbol = "&symbol="+row['symbol']
        query_url = url+function+key+symbol+output
        print(query_url)
        response = requests.get(query_url).json()
        time.sleep(13)
        print("Request completed, converting to dataframe and trying to push it to the database")
        result_dict=response["Time Series (Daily)"]
        results_df=pd.DataFrame(result_dict)
        results_df2 = results_df.transpose()
        results_df2["company"] = row['symbol']
        results_df2.reset_index(inplace=True)
        results_df2.columns=['stock_date', 'open', 'high', 'low', 'close', 'volume','stock_symbol']
        stock_type = {"stock_date": sqlalchemy.DateTime(), 
              "open": sqlalchemy.types.Float(precision=5, asdecimal=True), 
              "high": sqlalchemy.types.Float(precision=5, asdecimal=True), 
              "low": sqlalchemy.types.Float(precision=5, asdecimal=True),
              "close": sqlalchemy.types.Float(precision=5, asdecimal=True),
              "volume": sqlalchemy.types.BigInteger(),
              "stock_symbol": sqlalchemy.types.VARCHAR(length=10),
             }
        dcnt = results_df2['open'].count()
        if dcnt > 0:
#             if index == 0:
#                 results_df2.to_sql(name="stock_daily", con=engine, if_exists='replace', index=False, dtype=stock_type)
#             else:
            results_df2.to_sql(name="stock_daily", con=engine, if_exists='append', index=False, dtype=stock_type)
    except Exception as e:
        logger.error('Failed to retreive stock data for the symbol:'+ row['symbol'] + ' Error : ' + str(e))
        err_cnt += 1;
    print(time.time()-start_time)
    print(etime-time.time())
    sleep_sec = etime-time.time()
    if rec_cnt == 5:
        rec_cnt = 0
        if sleep_sec > 0:
            print(f"API 5 calls/minute limit reached...  sleeping for {sleep_sec} seconds")
            time.sleep(sleep_sec)
            etime = time.time() + 60
    if index > 498:
        print("Reached maximum limit of 500 rows, existing the for loop...")
        break

print(" ")
print(f"End of processing {index+1} records in {round((time.time()-start_time)/60,4)} minutes!!! ")
print(f"There are {err_cnt} records ends with error")
print(" ")


0 BKB 1
Retrieving Stock data from Alphavantage website for the symbol BKB
https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&apikey=S2Q31T0C29CO77S1&symbol=BKB&outputsize=full


Failed to retreive stock data for the symbol:BKB Error : 'Time Series (Daily)'


Request completed, converting to dataframe and trying to push it to the database
14.07074499130249
45.9292631149292
 
End of processing 1 records in 0.2345 minutes!!! 
There are 1 records ends with error
 


In [ ]:
err_cnt

In [ ]:
print(time.time(), time.time()-300)

In [ ]:
time.strftime('%H:%M:%S', time.localtime(time.time()))